Проанализируем представленный датасет компании из сферы E-com и найдем ответы на ряд поставленных вопросов.

In [5]:
import requests
from urllib.parse import urlencode
import pandas as pd
import plotly.express as px
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Спарсим датасеты с диска и соединим их

In [6]:
def get_reference(
    public_key,
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
):
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    return download_url

In [3]:
olist_customers_dataset = pd.read_csv(get_reference('https://disk.yandex.ru/d/FUi5uSd6BfG_ig'))
olist_orders_dataset = pd.read_csv(get_reference('https://disk.yandex.ru/d/t9Li4JOfgxuUrg'),
                                  parse_dates=['order_purchase_timestamp', 'order_approved_at',
                                               'order_delivered_carrier_date', 'order_delivered_customer_date',
                                               'order_estimated_delivery_date'])
olist_order_items_dataset = pd.read_csv(get_reference('https://disk.yandex.ru/d/Gbt-yAcQrOe3Pw'), 
                                        parse_dates=['shipping_limit_date'])

In [4]:
df = olist_customers_dataset.merge(olist_orders_dataset, on='customer_id').merge(olist_order_items_dataset, on='order_id')

In [7]:
df.shape

(112650, 18)

In [8]:
df.dtypes

customer_id                              object
customer_unique_id                       object
customer_zip_code_prefix                  int64
customer_city                            object
customer_state                           object
order_id                                 object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
order_item_id                             int64
product_id                               object
seller_id                                object
shipping_limit_date              datetime64[ns]
price                                   float64
freight_value                           float64
dtype: object

Какие же заказы считать покупкой? Посмотрим повнимательные на данные

In [9]:
df.isna().sum()

customer_id                         0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  15
order_delivered_carrier_date     1194
order_delivered_customer_date    2454
order_estimated_delivery_date       0
order_item_id                       0
product_id                          0
seller_id                           0
shipping_limit_date                 0
price                               0
freight_value                       0
dtype: int64

15 заказов были сформированы но не оплачен:

In [10]:
df.query("order_approved_at == 'NaT' & order_status == 'delivered'").reset_index(drop=True)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,0bf35cac6cc7327065da879e2d90fae8,c4c0011e639bdbcf26059ddc38bd3c18,13225,varzea paulista,SP,d77031d6a3c8a52f019764e68f211c69,delivered,2017-02-18 11:04:19,NaT,2017-02-23 07:23:36,2017-03-02 16:15:23,2017-03-22,1,02a79d79e818ad0be36cfc843a6af7ad,cb3dd9ce66268c7a3ca7241ac70ab58c,2017-02-25 10:04:19,28.99,10.96
1,1e101e0daffaddce8159d25a8e53f2b2,c8822fce1d0bfa7ddf0da24fff947172,27945,macae,RJ,12a95a3c06dbaec84bcfb0e2da5d228a,delivered,2017-02-17 13:05:55,NaT,2017-02-22 11:23:11,2017-03-02 11:09:19,2017-03-20,1,c6dd917a0be2a704582055949915ab32,7a67c85e85bb2ce8582c35f2203ad736,2017-02-21 12:05:55,79.99,15.77
2,d5de688c321096d15508faae67a27051,d49f3dae6bad25d05160fc17aca5942d,36400,conselheiro lafaiete,MG,7002a78c79c519ac54022d4f8a65e6e8,delivered,2017-01-19 22:26:59,NaT,2017-01-27 11:08:05,2017-02-06 14:22:19,2017-03-16,1,c3b271f47e73d0c9ccf1b43b7606c705,cca3071e3e9bb7d12640c9fbe2301306,2017-01-29 22:26:59,45.90,14.52
3,07a2a7e0f63fd8cb757ed77d4245623c,79af1bbf230a2630487975aa5d7d6220,37660,paraisopolis,MG,51eb2eebd5d76a24625b31c33dd41449,delivered,2017-02-18 15:52:27,NaT,2017-02-23 03:09:14,2017-03-07 13:57:47,2017-03-29,1,7868a64aa111bbb4f41f8e1146c0becb,cca3071e3e9bb7d12640c9fbe2301306,2017-02-28 14:52:27,59.90,17.16
4,68d081753ad4fe22fc4d410a9eb1ca01,2e0a2166aa23da2472c6a60c4af6f7a6,3573,sao paulo,SP,d69e5d356402adc8cf17e08b5033acfb,delivered,2017-02-19 01:28:47,NaT,2017-02-23 03:11:48,2017-03-02 03:41:58,2017-03-27,1,cae2e38942c8489d9d7a87a3f525c06b,cca3071e3e9bb7d12640c9fbe2301306,2017-02-26 01:28:47,149.80,13.63
5,d85919cb3c0529589c6fa617f5f43281,c094ac95fcd52f821809ec232a7a6956,95795,sao vendelino,RS,3c0b8706b065f9919d0505d3b3343881,delivered,2017-02-17 15:53:27,NaT,2017-02-22 11:31:30,2017-03-03 11:47:47,2017-03-23,1,db8ed3d08891d16a2438a67ab3acb740,004c9cd9d87a3c30c522c48c4fc07416,2017-02-22 14:53:27,133.99,23.20
6,74bebaf46603f9340e3b50c6b086f992,f79be7c08dd24b72d34634f1b89333a4,65110,sao jose de ribamar,MA,2babbb4b15e6d2dfe95e2de765c97bce,delivered,2017-02-18 17:15:03,NaT,2017-02-22 11:23:11,2017-03-03 18:43:43,2017-03-31,1,c6dd917a0be2a704582055949915ab32,7a67c85e85bb2ce8582c35f2203ad736,2017-02-22 16:15:03,79.99,26.82
7,684cb238dc5b5d6366244e0e0776b450,6ff8b0d7b35d5c945633b8d60165691b,11030,santos,SP,c1d4211b3dae76144deccd6c74144a88,delivered,2017-01-19 12:48:08,NaT,2017-01-25 14:56:50,2017-01-30 18:16:01,2017-03-01,1,5ab02ca028398131a5ae91401eb49788,80e6699fe29150b372a0c8a1ebf7dcc8,2017-01-23 12:48:08,39.99,14.52
8,a3d3c38e58b9d2dfb9207cab690b6310,5a4fa4919cbf2b049e72be460a380e5b,35620,abaete,MG,2eecb0d85f281280f79fa00f9cec1a95,delivered,2017-02-17 17:21:55,NaT,2017-02-22 11:42:51,2017-03-03 12:16:03,2017-03-20,1,4fd676d9c4723d475026e40aeae56957,5fd924b4836098a5be0ecf81ba054ce0,2017-02-21 16:21:55,135.00,19.23
9,2127dc6603ac33544953ef05ec155771,8a9a08c7ca8900a200d83cf838a07e0b,6708,cotia,SP,e04abd8149ef81b95221e88f6ed9ab6a,delivered,2017-02-18 14:40:00,NaT,2017-02-23 12:04:47,2017-03-01 13:25:33,2017-03-17,1,0e20a07ca1714df21f9b07ca3bf7c682,4e7c18b98d84e05cbae3ff0ff03846c2,2017-02-22 13:40:00,309.90,39.11


По данным заказам не был подтвержден факт оплаты, при этом они были доставлены покупателю и имеют соотвествующий статус. Дело, по всей видимости в технической ошибке.

In [11]:
df.query("order_delivered_carrier_date == 'NaT'").order_status.value_counts()

canceled       466
invoiced       359
processing     357
unavailable      7
approved         3
delivered        2
Name: order_status, dtype: int64

In [12]:
df.query("order_delivered_carrier_date == 'NaT'").query("order_status == 'delivered'")

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
98532,afeb16c7f46396c0ed54acb45ccaaa40,a2ac81ecc3704410ae240e74d4f0af40,13334,indaiatuba,SP,2aa91108853cecb43c84a5dc5b277475,delivered,2017-09-29 08:52:58,2017-09-29 09:07:16,NaT,2017-11-20 19:44:47,2017-11-14,1,44c2baf621113fa7ac95fa06b4afbc68,3f2af2670e104d1bcb54022274daeac5,2017-10-18 10:07:16,179.0,14.98
110621,e08caf668d499a6d643dafd7c5cc498a,d77cf4be2654aa70ef150f8bfec076a6,91330,porto alegre,RS,2d858f451373b04fb5c984a1cc2defaf,delivered,2017-05-25 23:22:43,2017-05-25 23:30:16,NaT,NaT,2017-06-23,1,30b5b5635a79548a48d04162d971848f,f9bbdd976532d50b7816d285a22bd01e,2017-06-04 23:30:16,179.0,15.00


Из тех заказов, что не были переданы в логистическую службу большая часть уже была отменена, при этом, имеются заказы, имеющие отметку "доставлено", из них, у одного нет только даты передачи логистичской службе, а у дрого ни ее, ни даты, когда заказ был доставлен.

Рассмотрим заказы со статусом 'unavailable'

In [13]:
df.query("order_delivered_carrier_date == 'NaT' & order_status == 'unavailable'")

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
3831,98bfbf477ed9f40a0264f5e527a446be,8aa802da0f525a68e7fae5e67929ebdd,18147,aracariguama,SP,54bb06e1ca86bd99ee2a8d6288bf4ede,unavailable,2016-10-06 13:38:39,2016-10-06 14:22:19,NaT,NaT,2016-12-12,1,bef13288a0f8b26d763411ced1dfa268,cf281cf8b7affbdfe751b29cc85580e1,2016-10-29 09:43:46,64.90,25.33
56915,96bbcbc8372c86da857d842dfda6bd72,e347a8d0bfec0255258cc913ed28d3a9,28010,campos dos goytacazes,RJ,4dd47e84e6b8ff4a63d0b8425e6d788e,unavailable,2016-10-06 23:49:18,2016-10-07 01:09:08,NaT,NaT,2016-11-24,1,2aa489c905e1d41d8c716a6d6912d8c2,620c87c171fb2a6dd6e8bb4dec959fc6,2016-10-11 01:09:09,299.90,13.68
67080,88c36fbd3df96094f90f0315e1d2dd90,e629cd3ff381ad0a234016fa7dd76f46,8610,suzano,SP,3c3ca08854ca922fe8e9cedfd6841c8a,unavailable,2016-10-08 12:13:38,2016-10-08 13:21:35,NaT,NaT,2016-11-30,1,ac6c24af9f913c9375a3ea77883bda6e,5206cc4bc2297c833e6061c49bf9c43f,2016-10-12 13:21:35,35.00,10.96
70163,4a06381959b6670756de02e07b83815f,b92a2e5e8a6eabcc80882c7d68b2c70b,6730,vargem grande paulista,SP,dc18a044b56ed174037ca164cdf2e921,unavailable,2016-10-05 14:16:28,2016-10-06 15:56:49,NaT,NaT,2016-11-29,1,faa268ecc8a8434cb4ebee091b3222d6,f0563bacf40c311f1c4d3d6b67b8a7a7,2016-10-21 16:18:52,1199.00,28.78
77832,7f51c513004ef3f35e834c142f286778,30fceee7cb0dbe6926f2b1128f2b3648,31910,belo horizonte,MG,2fd1c83dd4714cf3cf796fffb6c8de62,unavailable,2016-10-05 17:21:29,2016-10-06 15:55:40,NaT,NaT,2016-11-29,1,17c7ec33a1657924ad493e6978b4bf91,3d01d1c414c44b5943a09619ff3853cb,2016-10-10 15:55:42,99.00,9.12
77833,7f51c513004ef3f35e834c142f286778,30fceee7cb0dbe6926f2b1128f2b3648,31910,belo horizonte,MG,2fd1c83dd4714cf3cf796fffb6c8de62,unavailable,2016-10-05 17:21:29,2016-10-06 15:55:40,NaT,NaT,2016-11-29,2,966a067b38634929844120e39b1576bc,61f159ef6da2d441951d2c0efa719362,2016-10-10 15:55:41,59.99,24.08
104004,3d8e326c3cef3a9bba7ca121baac5fc3,b18b6d40c3c5510727e0f22b568d3e1f,91430,porto alegre,RS,1a47da1d66c70489c8e35fe2b5433ab7,unavailable,2016-10-07 18:16:31,2016-10-07 18:53:48,NaT,NaT,2016-12-07,1,eba7488e1c67729f045ab43fac426f2e,620c87c171fb2a6dd6e8bb4dec959fc6,2016-10-11 18:53:48,249.90,20.85


Как мы видим, данные заказы были оплачены, но до логистической службы они уже не дошли, возможно, в нашем приложении была неактуальная информация о наличии товаров из данного заказа

Исходя из вышеперечисленного я возьму следующие критерии покупки:
* Подтвержена оплата заказа (order_approved_at != 'NaT'), но сделаю исключение для тех 15 заказов, которые отмечались выше так как, судя по всему, заказы были доставлены и оплачены, но произошла ошибка или для данных заказав был предусмотрен особый порядок оплаты
* Есть время доставки товара (order_delivered_customer_date != 'NaT')
* Статус заказа - 'доставлено' (order_status == 'delivered') \
Приступим к заданиям

In [14]:
purchases = df.query("(order_approved_at != 'NaT' | order_approved_at == 'NaT' & order_status == 'delivered') \
                       & order_delivered_customer_date != 'NaT' & order_status == 'delivered'") # датасет с покупками

### 1. Узнаем, сколько у нас пользователей, которые совершили покупку только один раз

In [15]:
purchases.groupby('customer_unique_id') \
         .agg({'order_id': 'nunique'}) \
         .query("order_id == 1") \
         .shape[0]

90549

Таких пользователей 90_549

### 2. Посчитаем, сколько заказов в месяц в среднем не доставляется по разным причинам


In [16]:
df['order_year'] = df.order_estimated_delivery_date.dt.year
df['order_month'] = df.order_estimated_delivery_date.dt.month

In [17]:
temp = df.groupby(['order_year', 'order_month', 'order_status'], as_index=False) \
         .agg({'order_id': 'nunique'}) \
         .pivot(index=['order_year', 'order_month'], columns='order_status', values='order_id') \
         .loc[:, ['canceled', 'unavailable']] \
         .fillna(0).mean()

In [18]:
temp['total'] = temp.canceled + temp.unavailable

In [19]:
temp

order_status
canceled       18.44
unavailable     0.24
total          18.68
dtype: float64

Или мы можем узнать количество месяцев, когда должны были(или будут) поступить заказы в отчетном периоде:

In [20]:
df.order_estimated_delivery_date.dt.strftime('%Y-%m').nunique()

25

Узнать количество недоставленных

In [21]:
df.order_status.value_counts()

delivered      110197
shipped          1185
canceled          542
invoiced          359
processing        357
unavailable         7
approved            3
Name: order_status, dtype: int64

In [22]:
542 + 7

549

И поделить на количество месяцев

In [23]:
549 / 25

21.96

In [24]:
542 / 25 # canceled

21.68

In [25]:
7 / 25 # unavailable

0.28

### 3. По каждому товару определим, в какой день недели товар чаще всего покупается.

In [26]:
purchases['order_day'] = purchases['order_approved_at'].dt.day_name().dropna() # добавим колонку с днями недели
                                                                               # удалим те 15 значений, без даты оплаты

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
purchases.groupby(['product_id', 'order_day']) \
         .agg({'customer_id': 'count'}) \
         .rename(columns={'customer_id': 'number_of_orders'}) \
         .pivot_table(index='product_id', columns='order_day', values='number_of_orders') \
         .apply(lambda x: ', '.join(x[x.values == x.max()].index), axis=1)

product_id
00066f42aeeb9f3007548bb9d3f33c38               Sunday
00088930e925c41fd95ebfe695fd2655              Tuesday
0009406fd7479715e4bef61dd91f2462               Friday
000b8f95fcb9e0096488278317764d19    Friday, Wednesday
000d9be29b5207b54e86aa1b1ac54872              Tuesday
                                          ...        
fff6177642830a9a94a0f2cba5e476d1     Monday, Saturday
fff81cc3158d2725c0655ab9ba0f712c               Monday
fff9553ac224cec9d15d49f5a263411f             Saturday
fffdb2d0ec8d6a61f0a0a0db3f25b441              Tuesday
fffe9eeff12fcbd74a2f2b007dde0c58            Wednesday
Length: 32211, dtype: object

### 4. Выясним, сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)

In [28]:
purchases['order_year'] = purchases.order_approved_at.dt.year # колонка с годом 
purchases['order_month'] = purchases.order_approved_at.dt.month # с месяцем

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
purchases['week_in_month'] = round(purchases.order_approved_at.dt.days_in_month / 7, 2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
purchases_by_month = purchases.groupby(['customer_unique_id', 'order_year', 'order_month', 'week_in_month'], as_index=False) \
                              .agg({'order_id': 'nunique'}) \
                              .rename(columns={'order_id': 'number_of_purchases'})
purchases_by_month

,customer_unique_id,order_year,order_month,week_in_month,number_of_purchases
0,0000366f3b9a7992bf8c76cfdf3221e2,2018.0,5.0,4.43,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018.0,5.0,4.43,1
2,0000f46a3911fa3c0805444483337064,2017.0,3.0,4.43,1
3,0000f6ccb0745a6a4b88665a16c9f078,2017.0,10.0,4.43,1
4,0004aac84e0df4da2b147fca70cf8255,2017.0,11.0,4.29,1
...,...,...,...,...,...
95163,fffcf5a5ff07b0908bd4e2dbc735a684,2017.0,6.0,4.29,1
95164,fffea47cd6d3cc0a88bd621562a9d061,2017.0,12.0,4.43,1
95165,ffff371b4d645b6ecea244b27531430a,2017.0,2.0,4.00,1
95166,ffff5962728ec6157033ef9805bacc48,2018.0,5.0,4.43,1


In [31]:
purchases_by_month['purchases_mean'] = purchases_by_month.apply(lambda x: x.number_of_purchases / x.week_in_month, axis=1)
purchases_by_month

,customer_unique_id,order_year,order_month,week_in_month,number_of_purchases,purchases_mean
0,0000366f3b9a7992bf8c76cfdf3221e2,2018.0,5.0,4.43,1,0.225734
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018.0,5.0,4.43,1,0.225734
2,0000f46a3911fa3c0805444483337064,2017.0,3.0,4.43,1,0.225734
3,0000f6ccb0745a6a4b88665a16c9f078,2017.0,10.0,4.43,1,0.225734
4,0004aac84e0df4da2b147fca70cf8255,2017.0,11.0,4.29,1,0.233100
...,...,...,...,...,...,...
95163,fffcf5a5ff07b0908bd4e2dbc735a684,2017.0,6.0,4.29,1,0.233100
95164,fffea47cd6d3cc0a88bd621562a9d061,2017.0,12.0,4.43,1,0.225734
95165,ffff371b4d645b6ecea244b27531430a,2017.0,2.0,4.00,1,0.250000
95166,ffff5962728ec6157033ef9805bacc48,2018.0,5.0,4.43,1,0.225734


### 5. Используя pandas, проведем когортный анализ пользователей. В период с января по декабрь выявим когорту с самым высоким retention на 3й месяц. 

Определим, когда к нам пришел каждый из клиентов(первая покупка) 

In [32]:
first_purchase = purchases.groupby('customer_unique_id') \
                      .agg({'order_approved_at': 'min'}) \
                      .rename(columns={'order_approved_at': 'birthday'}) \
                      .birthday.dt.to_period('M')
first_purchase

customer_unique_id
0000366f3b9a7992bf8c76cfdf3221e2    2018-05
0000b849f77a49e4a4ce2b2a4ca5be3f    2018-05
0000f46a3911fa3c0805444483337064    2017-03
0000f6ccb0745a6a4b88665a16c9f078    2017-10
0004aac84e0df4da2b147fca70cf8255    2017-11
                                     ...   
fffcf5a5ff07b0908bd4e2dbc735a684    2017-06
fffea47cd6d3cc0a88bd621562a9d061    2017-12
ffff371b4d645b6ecea244b27531430a    2017-02
ffff5962728ec6157033ef9805bacc48    2018-05
ffffd2657e2aad2907e67c3e9daecbeb    2017-05
Name: birthday, Length: 93350, dtype: period[M]

In [33]:
purchases = pd.merge(purchases, first_purchase, on='customer_unique_id').dropna() # вмержим серию в датафрейм
                                                                                  # удалим те самые 15 заказов

Создадим колонку такого же формата с датами всех заказов

In [34]:
purchases['month_of_purchase'] = purchases.order_approved_at.dt.to_period('M')

Для каждой следующей покупки узнаем сколько времени прошло с самого первого заказа 

In [35]:
purchases['month_diff'] = purchases['month_of_purchase'].subtract(purchases['birthday']) \
                                                       .apply(lambda x: x.n)

Развернем полученные данные в сводной таблице

In [36]:
cohort_table = purchases.pivot_table(
                                      columns='month_diff',
                                      index='birthday',
                                      values='customer_unique_id',
                                      aggfunc='nunique')          

Найдем retention rate

In [37]:
retention_table = cohort_table.div(cohort_table[0], axis=0)
retention_table 

month_diff,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20
birthday,,,,,,,,,,,,,,,,,,,,
2016-09,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10,1.0,NaN,NaN,NaN,NaN,NaN,0.003817,NaN,NaN,0.003817,NaN,0.003817,NaN,0.003817,NaN,0.003817,NaN,0.003817,0.007634,0.007634
2016-12,1.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01,1.0,0.004392,0.002928,0.001464,0.004392,0.001464,0.004392,0.001464,0.001464,NaN,0.002928,0.002928,0.007321,0.004392,0.001464,0.001464,0.002928,0.004392,0.001464,NaN
2017-02,1.0,0.001861,0.002481,0.001861,0.004342,0.001241,0.002481,0.001861,0.000620,0.001861,0.001241,0.003102,0.001241,0.001861,0.001241,0.000620,0.000620,0.001861,NaN,NaN
2017-03,1.0,0.004381,0.003584,0.003584,0.003584,0.001593,0.001593,0.003186,0.002788,0.001195,0.003584,0.001195,0.001991,0.001195,0.001593,0.002389,0.000796,0.001195,NaN,NaN
2017-04,1.0,0.006272,0.002240,0.001792,0.002240,0.002688,0.003584,0.002688,0.003584,0.001792,0.002240,0.000896,0.000448,0.000448,0.000896,0.000896,0.001344,NaN,NaN,NaN
2017-05,1.0,0.005214,0.004635,0.002897,0.002897,0.003187,0.004056,0.001448,0.002607,0.002607,0.002897,0.003476,0.001738,0.000579,0.001738,0.002028,NaN,NaN,NaN,NaN
2017-06,1.0,0.004928,0.003942,0.004271,0.002957,0.003614,0.003614,0.002300,0.001314,0.001971,0.002957,0.003942,0.001643,0.001643,0.002300,NaN,NaN,NaN,NaN,NaN


Так как под понятие "период с января по декабрь" попадает только значения 17 года, оставим только их

In [38]:
final = retention_table.loc['2017-01': '2017-12']
final

month_diff,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20
birthday,,,,,,,,,,,,,,,,,,,,
2017-01,1.0,0.004392,0.002928,0.001464,0.004392,0.001464,0.004392,0.001464,0.001464,NaN,0.002928,0.002928,0.007321,0.004392,0.001464,0.001464,0.002928,0.004392,0.001464,NaN
2017-02,1.0,0.001861,0.002481,0.001861,0.004342,0.001241,0.002481,0.001861,0.000620,0.001861,0.001241,0.003102,0.001241,0.001861,0.001241,0.000620,0.000620,0.001861,NaN,NaN
2017-03,1.0,0.004381,0.003584,0.003584,0.003584,0.001593,0.001593,0.003186,0.002788,0.001195,0.003584,0.001195,0.001991,0.001195,0.001593,0.002389,0.000796,0.001195,NaN,NaN
2017-04,1.0,0.006272,0.002240,0.001792,0.002240,0.002688,0.003584,0.002688,0.003584,0.001792,0.002240,0.000896,0.000448,0.000448,0.000896,0.000896,0.001344,NaN,NaN,NaN
2017-05,1.0,0.005214,0.004635,0.002897,0.002897,0.003187,0.004056,0.001448,0.002607,0.002607,0.002897,0.003476,0.001738,0.000579,0.001738,0.002028,NaN,NaN,NaN,NaN
2017-06,1.0,0.004928,0.003942,0.004271,0.002957,0.003614,0.003614,0.002300,0.001314,0.001971,0.002957,0.003942,0.001643,0.001643,0.002300,NaN,NaN,NaN,NaN,NaN
2017-07,1.0,0.005392,0.003235,0.001887,0.003505,0.001887,0.003235,0.001078,0.001887,0.002427,0.002157,0.002966,0.001348,0.002427,NaN,NaN,NaN,NaN,NaN,NaN
2017-08,1.0,0.006861,0.003676,0.002695,0.003676,0.005146,0.002695,0.002695,0.001225,0.001715,0.002450,0.001715,0.001470,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-09,1.0,0.006710,0.004970,0.003231,0.004225,0.002734,0.002237,0.002237,0.002982,0.001740,0.002485,0.000746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
(final
.style
.set_caption('Retention by cohort')  # добавляем подпись
.background_gradient(cmap='viridis')  # раскрашиваем ячейки по столбцам
.highlight_null('white')  # делаем белый фон для значений NaN
.format("{:.2%}", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту

/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:2813: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:2814: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


month_diff,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20
birthday,,,,,,,,,,,,,,,,,,,,
2017-01,100.00%,0.44%,0.29%,0.15%,0.44%,0.15%,0.44%,0.15%,0.15%,,0.29%,0.29%,0.73%,0.44%,0.15%,0.15%,0.29%,0.44%,0.15%,
2017-02,100.00%,0.19%,0.25%,0.19%,0.43%,0.12%,0.25%,0.19%,0.06%,0.19%,0.12%,0.31%,0.12%,0.19%,0.12%,0.06%,0.06%,0.19%,,
2017-03,100.00%,0.44%,0.36%,0.36%,0.36%,0.16%,0.16%,0.32%,0.28%,0.12%,0.36%,0.12%,0.20%,0.12%,0.16%,0.24%,0.08%,0.12%,,
2017-04,100.00%,0.63%,0.22%,0.18%,0.22%,0.27%,0.36%,0.27%,0.36%,0.18%,0.22%,0.09%,0.04%,0.04%,0.09%,0.09%,0.13%,,,
2017-05,100.00%,0.52%,0.46%,0.29%,0.29%,0.32%,0.41%,0.14%,0.26%,0.26%,0.29%,0.35%,0.17%,0.06%,0.17%,0.20%,,,,
2017-06,100.00%,0.49%,0.39%,0.43%,0.30%,0.36%,0.36%,0.23%,0.13%,0.20%,0.30%,0.39%,0.16%,0.16%,0.23%,,,,,
2017-07,100.00%,0.54%,0.32%,0.19%,0.35%,0.19%,0.32%,0.11%,0.19%,0.24%,0.22%,0.30%,0.13%,0.24%,,,,,,
2017-08,100.00%,0.69%,0.37%,0.27%,0.37%,0.51%,0.27%,0.27%,0.12%,0.17%,0.25%,0.17%,0.15%,,,,,,,
2017-09,100.00%,0.67%,0.50%,0.32%,0.42%,0.27%,0.22%,0.22%,0.30%,0.17%,0.25%,0.07%,,,,,,,,


Итак, наибольший ретеншн на 3 месяц наблюдается в когорте 2017-06(0.43%)	

### 6. RFM-анализ

   В кластеризации выберем следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время.
  Для каждого RFM-сегмента построем границы метрик recency, frequency и monetary для интерпретации этих кластеров.

Пусть "сегодня" - это день следующий за ожидаемым днем доставки послднего заказа + 1 день

In [40]:
purchases.order_delivered_customer_date.max()

Timestamp('2018-10-17 13:22:46')

In [41]:
current_date = pd.Timestamp(2018, 10, 18, 10)

Создадим таблицу для RFM анализа

In [42]:
rfm = purchases.groupby('customer_unique_id') \
               .agg({'order_approved_at': lambda x: current_date - x.max(),
                     'order_id': 'nunique',
                     'price': 'sum'}) \
               .rename(columns={'order_approved_at': 'Recency',
                                'order_id': 'Frequency',
                                'price': 'Monetary'})
rfm

,Recency,Frequency,Monetary
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,160 days 22:48:42,1,129.90
0000b849f77a49e4a4ce2b2a4ca5be3f,163 days 15:34:16,1,18.90
0000f46a3911fa3c0805444483337064,586 days 12:54:57,1,69.00
0000f6ccb0745a6a4b88665a16c9f078,370 days 13:10:43,1,25.99
0004aac84e0df4da2b147fca70cf8255,337 days 13:53:08,1,180.00
...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,496 days 12:44:44,1,1570.00
fffea47cd6d3cc0a88bd621562a9d061,311 days 13:43:40,1,64.89
ffff371b4d645b6ecea244b27531430a,617 days 17:57:16,1,89.90


Попробуем подсчитать уникальные значения для каждой колонки

In [43]:
r_per_25, r_pre_50, r_pre_75 = np.percentile(rfm.Recency.unique(), [25, 50, 75])
r_per_25, r_pre_50, r_pre_75 = pd.to_timedelta(r_per_25), pd.to_timedelta(r_pre_50), pd.to_timedelta(r_pre_75)
print('25th percentile: ' + str(r_per_25), '\n50th percentile: ' + str(r_pre_50), '\n75th percentile: ' + str(r_pre_75))

25th percentile: 168 days 05:47:15 
50th percentile: 275 days 06:25:23 
75th percentile: 405 days 17:49:42


In [44]:
f_per_25, f_pre_50, f_pre_75 = np.percentile(rfm.Frequency.unique(), [25, 50, 75])
print('25th percentile: ' + str(f_per_25), '\n50th percentile: ' + str(f_pre_50), '\n75th percentile: ' + str(f_pre_75))

25th percentile: 3.0 
50th percentile: 5.0 
75th percentile: 7.0


In [45]:
m_per_25, m_pre_50, m_pre_75 = np.percentile(rfm.Monetary.unique(), [25, 50, 75])
print('25th percentile: ' + str(m_per_25), '\n50th percentile: ' + str(m_pre_50), '\n75th percentile: ' + str(m_pre_75))

25th percentile: 75.3 
50th percentile: 163.79 
75th percentile: 357.985


Поулченные квартили в столбце «Recency» не сильно отличаются от квартилей DataFrame, в то время как столбец Frequency приобрел разные значения для квартилей, значения столбца Monetary хоть и увеличились почти в два раза, но также применимы для сегментации пользователей. Поэтому мы будем использовать эти значения для сегментации

In [59]:
edges = [rfm.Recency.min(), r_per_25, r_pre_50, r_pre_75, rfm.Recency.max()]
rfm['R'], recency_bins = pd.cut(rfm.Recency,
                                edges,
                                labels=[1, 2, 3, 4],
                                include_lowest=True,
                                retbins=True)
rfm

,Recency,Frequency,Monetary,R,F,M,RFM,Segment
customer_unique_id,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,160 days 22:48:42,1,129.90,1,4,3,143,Unsteady customers
0000b849f77a49e4a4ce2b2a4ca5be3f,163 days 15:34:16,1,18.90,1,4,4,144,Unsteady customers
0000f46a3911fa3c0805444483337064,586 days 12:54:57,1,69.00,4,4,4,444,Lost customers
0000f6ccb0745a6a4b88665a16c9f078,370 days 13:10:43,1,25.99,3,4,4,344,Potential lost customers
0004aac84e0df4da2b147fca70cf8255,337 days 13:53:08,1,180.00,3,4,2,342,Customers at risk
...,...,...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,496 days 12:44:44,1,1570.00,4,4,1,441,Inactive customers
fffea47cd6d3cc0a88bd621562a9d061,311 days 13:43:40,1,64.89,3,4,4,344,Potential lost customers
ffff371b4d645b6ecea244b27531430a,617 days 17:57:16,1,89.90,4,4,3,443,Lost customers


In [58]:
edges = [rfm.Frequency.min(), f_per_25, f_pre_50, f_pre_75, rfm.Frequency.max()]
rfm['F'], frequency_bins = pd.cut(rfm.Frequency,
                                  edges,
                                  labels=[4, 3, 2, 1],
                                  include_lowest=True,
                                  retbins=True)
rfm

,Recency,Frequency,Monetary,R,F,M,RFM,Segment
customer_unique_id,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,160 days 22:48:42,1,129.90,1,4,3,143,Unsteady customers
0000b849f77a49e4a4ce2b2a4ca5be3f,163 days 15:34:16,1,18.90,1,4,4,144,Unsteady customers
0000f46a3911fa3c0805444483337064,586 days 12:54:57,1,69.00,4,4,4,444,Lost customers
0000f6ccb0745a6a4b88665a16c9f078,370 days 13:10:43,1,25.99,3,4,4,344,Potential lost customers
0004aac84e0df4da2b147fca70cf8255,337 days 13:53:08,1,180.00,3,4,2,342,Customers at risk
...,...,...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,496 days 12:44:44,1,1570.00,4,4,1,441,Inactive customers
fffea47cd6d3cc0a88bd621562a9d061,311 days 13:43:40,1,64.89,3,4,4,344,Potential lost customers
ffff371b4d645b6ecea244b27531430a,617 days 17:57:16,1,89.90,4,4,3,443,Lost customers


In [60]:
edges = [rfm.Frequency.min(), f_per_25, f_pre_50, f_pre_75, rfm.Frequency.max()]
rfm['F'], frequency_bins = pd.cut(rfm.Frequency,
                                  edges,
                                  labels=[4, 3, 2, 1],
                                  include_lowest=True,
                                  retbins=True)
rfm

,Recency,Frequency,Monetary,R,F,M,RFM,Segment
customer_unique_id,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,160 days 22:48:42,1,129.90,1,4,3,143,Unsteady customers
0000b849f77a49e4a4ce2b2a4ca5be3f,163 days 15:34:16,1,18.90,1,4,4,144,Unsteady customers
0000f46a3911fa3c0805444483337064,586 days 12:54:57,1,69.00,4,4,4,444,Lost customers
0000f6ccb0745a6a4b88665a16c9f078,370 days 13:10:43,1,25.99,3,4,4,344,Potential lost customers
0004aac84e0df4da2b147fca70cf8255,337 days 13:53:08,1,180.00,3,4,2,342,Customers at risk
...,...,...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,496 days 12:44:44,1,1570.00,4,4,1,441,Inactive customers
fffea47cd6d3cc0a88bd621562a9d061,311 days 13:43:40,1,64.89,3,4,4,344,Potential lost customers
ffff371b4d645b6ecea244b27531430a,617 days 17:57:16,1,89.90,4,4,3,443,Lost customers


In [49]:
edges = [rfm.Monetary.min(), m_per_25, m_pre_50, m_pre_75, rfm.Monetary.max()]
rfm['M'], monetary_bins = pd.cut(rfm.Monetary,
                                     edges,
                                     labels=[4, 3, 2, 1],
                                     include_lowest=True,
                                     retbins=True)
rfm

,Recency,Frequency,Monetary,R,F,M
customer_unique_id,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,160 days 22:48:42,1,129.90,1,4,3
0000b849f77a49e4a4ce2b2a4ca5be3f,163 days 15:34:16,1,18.90,1,4,4
0000f46a3911fa3c0805444483337064,586 days 12:54:57,1,69.00,4,4,4
0000f6ccb0745a6a4b88665a16c9f078,370 days 13:10:43,1,25.99,3,4,4
0004aac84e0df4da2b147fca70cf8255,337 days 13:53:08,1,180.00,3,4,2
...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,496 days 12:44:44,1,1570.00,4,4,1
fffea47cd6d3cc0a88bd621562a9d061,311 days 13:43:40,1,64.89,3,4,4
ffff371b4d645b6ecea244b27531430a,617 days 17:57:16,1,89.90,4,4,3


Опредлеим каждого пользователя в финальный сегмент

In [50]:
rfm['RFM'] = rfm['R'].astype(str) + \
             rfm['F'].astype(str) + \
             rfm['M'].astype(str)
rfm

,Recency,Frequency,Monetary,R,F,M,RFM
customer_unique_id,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,160 days 22:48:42,1,129.90,1,4,3,143
0000b849f77a49e4a4ce2b2a4ca5be3f,163 days 15:34:16,1,18.90,1,4,4,144
0000f46a3911fa3c0805444483337064,586 days 12:54:57,1,69.00,4,4,4,444
0000f6ccb0745a6a4b88665a16c9f078,370 days 13:10:43,1,25.99,3,4,4,344
0004aac84e0df4da2b147fca70cf8255,337 days 13:53:08,1,180.00,3,4,2,342
...,...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,496 days 12:44:44,1,1570.00,4,4,1,441
fffea47cd6d3cc0a88bd621562a9d061,311 days 13:43:40,1,64.89,3,4,4,344
ffff371b4d645b6ecea244b27531430a,617 days 17:57:16,1,89.90,4,4,3,443


Посмотрим на количство пользователей в каждом сегменте

In [51]:
rfm['RFM'].value_counts()

144    10544
244    10365
344     9951
444     9908
143     8348
243     8048
343     7720
443     6901
142     3993
242     3950
342     3777
442     3609
141     1676
241     1537
341     1517
441     1445
131       14
231        8
132        4
121        4
331        4
221        3
232        3
434        2
424        1
211        1
111        1
431        1
332        1
Name: RFM, dtype: int64

Введем определение для сегментов:
* Top customers - активны, покупки, большие суммы заказов
* Active customers  — постоянно приносят доход
* Emerging customers – потенциал для перекрестных и дополнительных продаж
* Unsteady customers – потенциал для перекрестных и дополнительных продаж в зависимости от предложения
* ustomers at risk –  неактивные, но довольно часто совершают покупки и приносят высокий доход
* Потенциально потерянные клиенты:
* Inactive customers – неактивно, редко совершают покупки, но приносят высокие доходы
* Lost customers - неактивные, редко совершают покупки, приносят низкие доходы

In [52]:
segments = {'Top customers': ['111', '121', '112', '122'],
            'Active customers': ['131', '132', '113', '123', '133', '114', '124', '134'],
            'Emerging customers': ['211', '221', '231', '212', '222', '232'],
            'Unsteady customers': ['141', '142', '143', '144', '241', '242', '213', '223', '233',
                                   '243', '214', '224', '234', '244'],
            'Customers at risk': ['311', '312', '313', '314', '321', '322', '323', '324', '331',
                                  '332', '341', '342', '411', '412', '413', '414', '421', '422', '423', '424'],
            'Potential lost customers': ['333', '334', '343', '344'],
            'Inactive customers': ['441', '442'],
            'Lost customers': ['443', '444']}

In [53]:
def segmentation(rfm):
    for i, v in segments.items():
        if rfm in v:
            return i

In [54]:
rfm['Segment'] = rfm['RFM'].apply(segmentation)
rfm

,Recency,Frequency,Monetary,R,F,M,RFM,Segment
customer_unique_id,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,160 days 22:48:42,1,129.90,1,4,3,143,Unsteady customers
0000b849f77a49e4a4ce2b2a4ca5be3f,163 days 15:34:16,1,18.90,1,4,4,144,Unsteady customers
0000f46a3911fa3c0805444483337064,586 days 12:54:57,1,69.00,4,4,4,444,Lost customers
0000f6ccb0745a6a4b88665a16c9f078,370 days 13:10:43,1,25.99,3,4,4,344,Potential lost customers
0004aac84e0df4da2b147fca70cf8255,337 days 13:53:08,1,180.00,3,4,2,342,Customers at risk
...,...,...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,496 days 12:44:44,1,1570.00,4,4,1,441,Inactive customers
fffea47cd6d3cc0a88bd621562a9d061,311 days 13:43:40,1,64.89,3,4,4,344,Potential lost customers
ffff371b4d645b6ecea244b27531430a,617 days 17:57:16,1,89.90,4,4,3,443,Lost customers


In [55]:
rfm['Segment'].value_counts()

Unsteady customers          48461
Potential lost customers    17671
Lost customers              16809
Customers at risk            5300
Inactive customers           5054
Active customers               18
Emerging customers             15
Top customers                   5
Name: Segment, dtype: int64

посмотрим на среднее значение и медиану новых сегментов наших пользователей

In [56]:
rfm.groupby('Segment')[['Recency', 'Frequency', 'Monetary']].mean(numeric_only=False)

,Recency,Frequency,Monetary
Segment,,,
Active customers,107 days 22:27:46.722222222,4.166667,861.718889
Customers at risk,335 days 00:30:37.119245280,1.079245,367.357928
Emerging customers,217 days 12:24:01.666666668,5.200000,588.934000
Inactive customers,508 days 04:18:23.415314600,1.057182,376.970784
Lost customers,506 days 11:02:40.938128384,1.016301,70.881322
Potential lost customers,334 days 17:05:54.954558316,1.017260,73.380548
Top customers,101 days 15:57:27.400000,8.200000,690.602000
Unsteady customers,164 days 20:22:57.267906150,1.034378,141.345339


In [57]:
rfm.groupby('Segment')[['Recency', 'Frequency', 'Monetary']].median(numeric_only=False)

,Recency,Frequency,Monetary
Segment,,,
Active customers,118 days 03:01:43,4.0,534.475
Customers at risk,328 days 06:49:48.500000,1.0,255.000
Emerging customers,210 days 14:49:36,5.0,547.140
Inactive customers,502 days 14:34:45,1.0,254.900
Lost customers,498 days 20:17:16,1.0,61.900
Potential lost customers,327 days 18:04:15,1.0,66.900
Top customers,112 days 08:48:39,7.0,687.760
Unsteady customers,167 days 02:08:07,1.0,89.900
